### Ladi_ai_RAG

- API DOCS : http://43.202.173.71:3000/
- TASK : 루틴 추천
- 루틴 관련 데이터 저장(vectorDB) -> 검색(Retrieve) -> LLM(gpt-4o)의 Context로 사용

## LADI API

In [23]:
import requests
import json

# API 엔드포인트 URL
api_url = "http://43.202.173.71:3000/api/endpoint"  # 실제 엔드포인트 URL로 변경해야 합니다

# 요청 데이터
request_data = {
    "Difficult": "의욕 떨어짐, 목표 너무 큼",
    "Job": "학생",
    "gender": "남성",
    "age": 24,
    "goal": "꾸준히 운동하기"
}

# POST 요청 보내기
response = requests.post(api_url, json=request_data)

# 응답 처리
if response.status_code == 200:
    # 성공적인 응답
    response_data = response.json()
    print("API 응답:")
    print(json.dumps(response_data, indent=2, ensure_ascii=False))
else:
    # 오류 응답
    print(f"오류 발생: {response.status_code}")
    print(response.text)

오류 발생: 404
<!DOCTYPE html><html><head><title></title><link rel="stylesheet" href="/stylesheets/style.css"></head><body><h1>Not Found</h1><h2>404</h2><pre>NotFoundError: Not Found
    at file:///home/ubuntu/life-designer-server/app.js:39:8
    at Layer.handle [as handle_request] (/home/ubuntu/life-designer-server/node_modules/express/lib/router/layer.js:95:5)
    at trim_prefix (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:317:13)
    at /home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:284:7
    at Function.process_params (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:335:12)
    at next (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:275:10)
    at /home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:635:15
    at next (/home/ubuntu/life-designer-server/node_modules/express/lib/router/index.js:260:14)
    at Function.handle (/home/ubuntu/life-designer-s

### API

In [3]:
import requests

# API 엔드포인트 URL
token_url = "http://43.202.173.71:3000/auth/token"

# 요청 데이터 (필요한 경우 수정)
request_data = {
    "username": "your_username",  # 사용자 이름
    "password": "your_password"   # 비밀번호
}

# POST 요청 보내기
response = requests.post(token_url)

# 응답 처리
if response.status_code == 200:
    # 성공적인 응답
    token_data = response.json()
    print("토큰 응답:")
    print(json.dumps(token_data, indent=2, ensure_ascii=False))
else:
    # 오류 응답
    print(f"오류 발생: {response.status_code}")
    print(response.text)


오류 발생: 401
Unauthorized


In [4]:
import requests
# response=requests.get('http://43.202.173.71:3000/statistics')
# print(response.json())

response = requests.get('http://43.202.173.71:3000/statistics')
if response.status_code == 200:
    try:
        data = response.json()
    except ValueError:
        print("응답이 JSON 형식이 아닙니다:", response.text)
else:
    print(f"오류 발생: {response.status_code} - {response.text}")


오류 발생: 401 - Unauthorized


In [ ]:
from flask import Flask, jsonify
import requests

app = Flask(__name__)

@app.route('/get_statistics', methods=['GET'])
def get_statistics():
    try:
        # localhost:3000/statistics에 GET 요청 보내기
        response = requests.get('http://43.202.173.71:3000/statistics')
        
        # 응답 상태 코드 확인
        if response.status_code == 200:
            # 성공적으로 데이터를 받아왔을 때
            data = response.json()
            return jsonify(data), 200
        else:
            # 오류 발생 시
            return jsonify({"error": "외부 API에서 데이터를 가져오는 데 실패했습니다."}), response.status_code
    
    except requests.RequestException as e:
        # 네트워크 오류 등 예외 처리
        return jsonify({"error": f"요청 중 오류 발생: {str(e)}"}), 500

if __name__ == '__main__':
    app.run(debug=True, port=5001)

## RAG Setting

### Json_Loader

In [24]:
import json
from langchain.schema import Document
from pprint import pprint

# JSON 파일 로드 함수 정의
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# JSON 데이터 로드
data = load_json_file("data/dummy_data.json")
\
# 필요한 정보 추출 및 Document 객체 생성
docs = [
    Document(
        page_content=item["context"],
        metadata={
            "id": item["id"],
            "question": item["question"],
            "age": item["age"],
            "gender": item["gender"],
            "Job": item["Job"],
            "Difficult": item["Difficult"],
            "goal": item["goal"]
        }
    )
    for item in data
]

# 결과 출력
pprint(docs)

[Document(metadata={'id': 1, 'question': '효과적인 운동 방법은 무엇인가요?', 'age': 18, 'gender': '남성', 'Job': '직장인', 'Difficult': '체력 부족, 운동에 대한 두려움', 'goal': '스트레스 해소하기'}, page_content='운동 루틴에 대한 정보가 포함된 문서 내용입니다.'),
 Document(metadata={'id': 2, 'question': '운동 목표에 맞는 루틴은 무엇인가요?', 'age': 36, 'gender': '남성', 'Job': '프리랜서', 'Difficult': '의욕 떨어짐, 목표 너무 큼', 'goal': '스트레스 해소하기'}, page_content='운동 루틴에 대한 정보가 포함된 문서 내용입니다.'),
 Document(metadata={'id': 3, 'question': '운동 루틴에 대해 설명해주세요.', 'age': 22, 'gender': '여성', 'Job': '직장인', 'Difficult': '운동 후 피로감', 'goal': '스트레스 해소하기'}, page_content='운동 루틴에 대한 정보가 포함된 문서 내용입니다.'),
 Document(metadata={'id': 4, 'question': '추천하는 운동 루틴은 무엇인가요?', 'age': 52, 'gender': '남성', 'Job': '주부', 'Difficult': '운동에 대한 정보 부족', 'goal': '스트레스 해소하기'}, page_content='효과적인 운동 방법에 대한 정보입니다.'),
 Document(metadata={'id': 5, 'question': '추천하는 운동 루틴은 무엇인가요?', 'age': 38, 'gender': '남성', 'Job': '학생', 'Difficult': '운동 후 피로감', 'goal': '스트레스 해소하기'}, page_content='운동을 통해 건강을 유지하는 방법입니다.'),
 Document(m

### Embedding and Vectorstore

In [25]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_upstage import UpstageEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 1. 벡터 저장소 생성
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")
vectorstore = Chroma(embedding_function=passage_embeddings)


### text_spliting and add_documents

In [26]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

fs = LocalFileStore("./cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    passage_embeddings, fs, namespace=passage_embeddings.model
)

# 2. 문서 임베딩 및 저장
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
vectorstore.add_documents(texts)


['64ac78f6-0126-44cb-bdc3-5e67663b1378',
 '80fd2f05-096f-4a06-bb78-9c6c6785f200',
 'b54b10ac-8dc2-4b2b-a553-ae1e9cc60010',
 '745df946-2dbb-4eeb-9c66-3081565698bb',
 '11f85b26-8e8e-4eeb-ae40-69555594f95a',
 '20eec00d-59fe-4d50-a381-ede4ced25693',
 'ce288103-baf2-4bfd-a4b5-2624ff142d9f',
 '81815b5a-b58f-435b-a646-b4aa5491cca2',
 'ef0370c0-4895-4b9b-8284-78b186ed6ccf',
 'c3a6bc51-60e7-434c-94f1-8addb79f414a',
 '82649352-fe66-49d0-ae3c-d97e901ba3d1',
 '548d03fe-40d4-4b62-b49f-becf89452b60',
 '47febfa4-ae80-4d0a-b4bd-d6db37381c9a',
 '71bd1e6d-aee7-4b59-9dc4-5a687b3e6a15',
 '72728908-f14e-4eff-8e53-34db1b3dc3a6',
 '7acc7140-ee38-4719-9201-39031e6a7ec5',
 '7fe86e3a-3c4c-4544-b7ce-405b6610fc5b',
 '6800a60f-acab-4ddd-8269-b386ac936070',
 'e984c5c8-f51c-4d67-9bd8-be743710c2b1',
 '25d44ddd-5438-41b4-a8b6-71c2de38ac05']

### 더미 데이터 생성

In [42]:
import json
import random

# 더미 데이터 생성 함수
def generate_dummy_data():
    contexts = [
        "운동 루틴에 대한 정보가 포함된 문서 내용입니다.",
        "다양한 운동 루틴을 추천합니다.",
        "운동 목표에 맞는 루틴을 제안합니다.",
        "효과적인 운동 방법에 대한 정보입니다.",
        "운동을 통해 건강을 유지하는 방법입니다."
    ]
    
    questions = [
        "운동 루틴에 대해 설명해주세요.",
        "추천하는 운동 루틴은 무엇인가요?",
        "어떤 운동을 해야 할까요?",
        "운동 목표에 맞는 루틴은 무엇인가요?",
        "효과적인 운동 방법은 무엇인가요?"
    ]
    
    ages = [random.randint(18, 60) for _ in range(20)]
    genders = ["남성", "여성"]
    jobs = ["학생", "직장인", "주부", "프리랜서"]
    difficulties = [
        "의욕 떨어짐, 목표 너무 큼",
        "시간 부족, 운동 습관 없음",
        "운동에 대한 정보 부족",
        "체력 부족, 운동에 대한 두려움",
        "운동 후 피로감"
    ]
    goals = [
        "꾸준히 운동하기",
        "체중 감량하기",
        "근력 키우기",
        "유연성 향상하기",
        "스트레스 해소하기"
    ]
    
    dummy_data_list = []
    
    for i in range(20):
        dummy_data = {
            "id": i + 1,  # id 추가
            "context": random.choice(contexts),
            "question": random.choice(questions),
            "age": random.choice(ages),
            "gender": random.choice(genders),
            "Job": random.choice(jobs),
            "Difficult": random.choice(difficulties),
            "goal": random.choice(goals)
        }
        dummy_data_list.append(dummy_data)
    
    return dummy_data_list

# 더미 데이터 생성
dummy_data_list = generate_dummy_data()

# JSON 파일로 저장
with open('dummy_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(dummy_data_list, json_file, ensure_ascii=False, indent=2)

print("더미 데이터가 'dummy_data.json' 파일로 저장되었습니다.")

더미 데이터가 'dummy_data.json' 파일로 저장되었습니다.


### generate one

In [5]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional

class SubRoutine(BaseModel):
    """운동 루틴을 추천합니다."""
    emoji: str
    routineDetail: str
    minuteDuration: str

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0,response_format=SubRoutine)
response = llm.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")
# structured_llm = llm.with_structured_output(
#     SubRoutine,
#     method="json_schema",
#     include_raw=False,
#     strict=True
# )

# response = structured_llm.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")
import json

content = json.loads(response.content)
json_response = json.dumps(content, indent=2, ensure_ascii=False)
print(json_response)
print(type(json_response))


/Users/minu/Library/Caches/pypoetry/virtualenvs/langchain-kr-0guSvvrv-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)
Failed to use model_dump to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.model_dump() missing 1 required positional argument: 'self'")
Failed to use dict to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.dict() missing 1 required positional argument: 'self'")


{
  "emoji": "💪",
  "routineDetail": "1. 워밍업 (5분): 가벼운 조깅 또는 제자리에서 뛰기.\n2. 스쿼트 (3세트, 각 12회) - 하체 근육 강화.\n3. 푸시업 (3세트, 각 10회) - 상체 근육 강화.\n4. 플랭크 (3세트, 각 30초 유지) - 코어 근육 강화.\n5. 점핑잭 (3세트, 각 30초) - 유산소 운동.\n6. 쿨다운 (5분): 스트레칭으로 마무리.",
  "minuteDuration": "30분"
}
<class 'str'>


Failed to use model_dump to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.model_dump() missing 1 required positional argument: 'self'")
Failed to use dict to serialize <class 'pydantic._internal._model_construction.ModelMetaclass'> to JSON: TypeError("BaseModel.dict() missing 1 required positional argument: 'self'")
Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


### LLM Answer

In [27]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate

# 5. RAG 파이프라인 구축
prompt_template = PromptTemplate(
    input_variables=["context", "question", "age", "gender", "Job", "Difficult", "goal"],
        template="""당신은 API 서버입니다.
{context}를 참고하여 사용자에 맞는 루틴을 추천해주세요.
사용자의 나이는 {age}입니다.
사용자의 성별은 {gender}입니다.
사용자의 직업은 {job}입니다.
사용자의 겪고 있는 어려움은 {difficult}입니다.
사용자의 목표는 {goal}입니다.

질문: {question}
답변:
응답 형식
{{
  "subRoutine": 
    }}
      "emoji": "이모지",
      "routineDetail": "루틴 설명",
      "miniuteDuration": "시간(분 단위)"
    }}
  
}}
"""
)

retriever = vectorstore.as_retriever()



class SubRoutine(BaseModel):
    """운동 루틴을 추천합니다."""
    emoji: str = Field(description="이모지")
    routineDetail: str = Field(description="루틴 설명")
    minuteDuration: str = Field(description="시간(분 단위)")
#llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0,response_format=SubRoutine)
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.7)
parser = JsonOutputParser(pydantic_object=SubRoutine)
from langchain_core.output_parsers import StrOutputParser
parser= StrOutputParser()


retrieved_docs = retriever.invoke("30분 동안 할 수 있는 운동 루틴을 추천해주세요.")


In [28]:

# RAG 체인 생성
from operator import itemgetter
chain = (
    {
        "question":itemgetter("question"),
        "context":itemgetter("context"),
        "age":itemgetter("age"),
        "gender":itemgetter("gender"),
        "job":itemgetter("job"),
        "difficult":itemgetter("difficult"),
        "goal":itemgetter("goal")
    }
    | prompt_template
    | llm
    | parser
)

# 결과 생성
results = []
for i in range(5):

    result = chain.invoke(
        {"context":retrieved_docs,"question":"30분 동안 할 수 있는 운동 루틴을 추천해주세요.",
         "age":itemgetter("age"),"gender":itemgetter("gender"),
         "job":itemgetter("job"),"difficult":itemgetter("difficult"),
         "goal":itemgetter("goal")}
    )
    results.append(result)

# 결과 출력
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result)
    print()

Result 1:
사용자의 정보를 바탕으로 30분 동안 할 수 있는 운동 루틴을 추천드립니다.

### 추천 운동 루틴 (30분)

#### 스트레스 해소를 원하는 56세 남성 직장인 (의욕 저하, 목표가 큼)

1. **워밍업 (5분)**
   - 가벼운 스트레칭 (전신)
   - 제자리에서 1분간 걷기

2. **유산소 운동 (10분)**
   - 점핑 잭: 2분
   - 제자리에서 뛰기: 2분
   - 사이드 스텝: 2분
   - 스쿼트: 2분
   - 휴식: 2분

3. **근력 운동 (10분)**
   - 팔굽혀펴기: 2세트 x 10회
   - 플랭크: 30초 유지, 2회
   - 덤벨이나 물병을 이용한 이두근 운동: 2세트 x 12회
   - 휴식: 1분

4. **쿨다운 (5분)**
   - 가벼운 스트레칭 (전신)
   - 심호흡: 1분

---

#### 근력 키우기를 원하는 44세 남성 프리랜서 (운동 후 피로감)

1. **워밍업 (5분)**
   - 가벼운 스트레칭 (특히 하체)
   - 제자리에서 1분간 걷기

2. **근력 운동 (20분)**
   - 스쿼트: 3세트 x 10회
   - 푸쉬업: 3세트 x 8회
   - 런지: 3세트 x 10회 (양쪽 다리)
   - 덤벨 또는 물병을 이용한 숄더 프레스: 3세트 x 10회
   - 휴식: 세트 간 1분

3. **쿨다운 (5분)**
   - 가벼운 스트레칭 (특히 사용한 근육)
   - 심호흡: 1분

이 루틴은 각 사용자의 목표에 맞춰 조정되었습니다. 운동 후 피로감이 느껴질 경우, 운동 강도를 조절하거나 충분한 휴식을 취하는 것이 중요합니다.

Result 2:
사용자의 정보를 바탕으로 적절한 운동 루틴을 추천하겠습니다. 사용자 정보는 다음과 같습니다:

- 나이: 56세
- 성별: 남성
- 직업: 직장인
- 겪고 있는 어려움: 의욕 떨어짐, 목표 너무 큼
- 목표: 스트레스 해소하기

### 30분 운동 루틴 추천

1. **워밍업 (5분)**  
   - 가벼운 스트레칭: 목,